<a href="https://colab.research.google.com/github/Osw1997/An-Introduction-to-Statistical-Learning/blob/main/Scripts/Resampling%20methods/applied_exercises_ch5_resampling_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 5 - Resampling methods

In [1]:
!pip install --upgrade scipy
!pip install --upgrade plotly

     |████████████████████████████████| 38.1 MB 1.2 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 28.8 MB 86.1 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


# Applied exercises
---
---
---
## Exercise 5.
In Chapter 4, we used logistic regression to predict the probability of
default using income and balance on the Default data set. We will
now estimate the test error of this logistic regression model using the
validation set approach. Do not forget to set a random seed before
beginning your analysis.


In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t

In [3]:
import scipy
scipy.__version__

'1.7.3'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
default = pd.read_csv('drive/MyDrive/itsl_datasets/Default.csv')
default.columns

Index(['default', 'student', 'balance', 'income'], dtype='object')

In [6]:
default.describe()

,balance,income
count,10000.000000,10000.000000
mean,835.374886,33516.981876
std,483.714985,13336.639563
min,0.000000,771.967729
25%,481.731105,21340.462903
50%,823.636973,34552.644802
75%,1166.308386,43807.729272
max,2654.322576,73554.233495


In [7]:
default.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [8]:
# One Hot Encoding
default_ohe = default.copy()
default_ohe['default'] = pd.get_dummies(default_ohe['default']).drop('No', axis=1)
default_ohe['student'] = pd.get_dummies(default_ohe['student']).drop('No', axis=1)
default_ohe.describe()

,default,student,balance,income
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.033300,0.294400,835.374886,33516.981876
std,0.179428,0.455795,483.714985,13336.639563
min,0.000000,0.000000,0.000000,771.967729
25%,0.000000,0.000000,481.731105,21340.462903
50%,0.000000,0.000000,823.636973,34552.644802
75%,0.000000,1.000000,1166.308386,43807.729272
max,1.000000,1.000000,2654.322576,73554.233495


In [9]:
default_ohe.head()

,default,student,balance,income
0,0,0,729.526495,44361.625074
1,0,1,817.180407,12106.134700
2,0,0,1073.549164,31767.138947
3,0,0,529.250605,35704.493935
4,0,0,785.655883,38463.495879


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
# (a) Fit a logistic regression model that uses income and balance to predict default.
print("=" * 50)
print("Logistic Regression for 'default' ~ 'income' + 'balance'")
SEED = 9
lr = LogisticRegression(random_state=SEED)
lr.fit(X=default_ohe[['income', 'balance']],
       y=default_ohe['default'])
print("Score for only one block: ", end=' ')
print(lr.score(X=default_ohe[['income', 'balance']],
       y=default_ohe['default']))


# (b) Using the validation set approach, estimate the test error of this
# model. In order to do this, you must perform the following steps:
# * i. Split the sample set into a training set and a validation set.
# * ii. Fit a multiple logistic regression model using only the training observations.
# * iii. Obtain a prediction of default status for each individual in
# the validation set by computing the posterior probability of
# default for that individual, and classifying the individual to
# the default category if the posterior probability is greater
# than 0.5.
# * iv. Compute the validation set error, which is the fraction of
# the observations in the validation set that are misclassified.
folds = 5
scores = cross_val_score(estimator=lr, 
                         X=default_ohe[['income', 'balance']],
                         y=default_ohe['default'],
                         cv=folds)
print(f"Scores for {folds}-Fold: ", end=' ')
print(scores)


# (c) Repeat the process in (b) three times, using three different splits
# of the observations into a training set and a validation set. Comment on the results obtained.
folds = 3
scores = cross_val_score(estimator=lr, 
                         X=default_ohe[['income', 'balance']],
                         y=default_ohe['default'],
                         cv=folds)
print(f"Scores for {folds}-Fold: ", end=' ')
print(scores)
print(f"mean scores: {sum(scores) / len(scores)}")


# (d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable
# for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a
# dummy variable for student leads to a reduction in the test error
# rate.
print()
print("=" * 50)
print("Logistic Regression for 'default' ~ 'income' + 'balance' + 'student' [DUMMY]")
folds = 5
scores = cross_val_score(estimator=lr, 
                         X=default_ohe[['income', 'balance', 'student']],
                         y=default_ohe['default'],
                         cv=folds)
print(f"Scores for {folds}-Fold: ", end=' ')
print(scores)
print(f"mean scores: {sum(scores) / len(scores)}")

# print("\n")
# print("=" * 50)
# print("Logistic Regression for 'default' ~ 'income' + 'balance' + 'student' [NOT DUMMY]")
# folds = 5
# scores = cross_val_score(estimator=lr, 
#                          X=default[['income', 'balance', 'student']],
#                          y=default['default'],
#                          cv=folds)
# print(f"Scores for {folds}-Fold: ", end=' ')
# print(scores)

Logistic Regression for 'default' ~ 'income' + 'balance'
Score for only one block:  0.9737
Scores for 5-Fold:  [0.9745 0.9735 0.9665 0.9655 0.975 ]
Scores for 3-Fold:  [0.97360528 0.97149715 0.96579658]
mean scores: 0.9702996694390494

Logistic Regression for 'default' ~ 'income' + 'balance' + 'student' [DUMMY]
Scores for 5-Fold:  [0.975  0.9685 0.9685 0.9675 0.9645]
mean scores: 0.9688000000000001


---
---
---
## Ejercicio 6
We continue to consider the use of a logistic regression model to
predict the probability of default using income and balance on the
Default data set. In particular, we will now compute estimates for
the standard errors of the income and balance logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using
the standard formula for computing the standard errors in the glm()
function. Do not forget to set a random seed before beginning your
analysis.



In [12]:
# (a) Using the summary() and glm() functions, determine the estimated standard
#  errors for the coefficients associated with income and balance in a multiple 
#  logistic regression model that uses both predictors.
from sklearn.linear_model import TweedieRegressor
print("Let's compare standard errors: ")
print(default_ohe[['income', 'balance']].describe())            # basic SE

glm = TweedieRegressor(power=0)
glm.fit(X=default_ohe[['income', 'balance']], y=default_ohe['default'])
glm.get_params()


print()
import statsmodels.api as sm
model = sm.formula.glm("default ~ income + balance - 1",
                       family=sm.families.Binomial(), data=default_ohe).fit()
print(model.summary())




Let's compare standard errors: 
             income       balance
count  10000.000000  10000.000000
mean   33516.981876    835.374886
std    13336.639563    483.714985
min      771.967729      0.000000
25%    21340.462903    481.731105
50%    34552.644802    823.636973
75%    43807.729272   1166.308386
max    73554.233495   2654.322576

                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9998
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1734.6
Date:                Thu, 07 Apr 2022   Deviance:                       3469.1
Time:                        01:20:42   Pearson chi2:                 3.87e+04
No. Iterations:              

In [70]:
# (b) Write a function, boot.fn(), that takes as input the Default data
# set as well as an index of the observations, and that outputs
# the coefficient estimates for income and balance in the multiple
# logistic regression model.
from scipy.stats import bootstrap
import numpy as np
# res = bootstrap((np.array(default_ohe[['income', 'balance']]),), np.std,
#                 confidence_level=0.95, random_state=SEED)
res_mlr = bootstrap((np.array(lr.predict(X=default_ohe[['income', 'balance']])),), np.std,
                confidence_level=0.95, random_state=SEED)
res_mlr.confidence_interval


# (c) Use the boot() function together with your boot.fn() function to
# estimate the standard errors of the logistic regression coefficients
# for income and balance.
res_predictors = bootstrap((np.array(default_ohe[['income', 'balance']]),), np.std,
                confidence_level=0.95, random_state=SEED, n_resamples=2500)
print("Standard errors for predictors: ", end=' ')
print(res_predictors.standard_error)


# (d) Comment on the estimated standard errors obtained using the
# glm() function and using your bootstrap function.
res_glm = bootstrap((np.array(model.predict()),), np.std,
                confidence_level=0.95, random_state=SEED, n_resamples=2500)
print("Standard errors for GLM output: ", end=' ')
print(res_glm.standard_error)

Standard errors for predictors:  [68.0697185   3.13708541]
Standard errors for GLM output:  0.0009011944428032815


array([0.00503124, 0.23310583, 0.02761586, ..., 0.00087821, 0.01840651,
       0.11497755])

---
---
---
# Exercise 7
In Sections 5.3.2 and 5.3.3, we saw that the cv.glm() function can be
used in order to compute the LOOCV test error estimate. Alternatively, one could compute those quantities using just the glm() and
predict.glm() functions, and a for loop. You will now take this approach in order to compute the LOOCV error for a simple logistic
regression model on the Weekly data set. Recall that in the context
of classification problems, the LOOCV error is given in (5.4).


In [73]:
# Transforming the Weekly dataset
weekly = pd.read_csv('drive/MyDrive/itsl_datasets/Weekly.csv')
weekly_ohe = weekly.copy()
weekly_ohe['Direction'] = pd.get_dummies(weekly_ohe['Direction']).drop('Down', axis=1)

In [186]:
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
# (a) Fit a logistic regression model that predicts Direction using Lag1 and Lag2.
log_reg = LogisticRegression()
log_reg.fit(X=weekly_ohe[['Lag1', 'Lag2']], y=weekly_ohe['Direction'])



# (b) Fit a logistic regression model that predicts Direction using Lag1
# and Lag2 using all but the first observation.
# loocv = LeaveOneOut()
# print(loocv.get_n_splits(np.array(weekly_ohe[['Lag1', 'Lag2']])))
# scores = list()
X = np.array(weekly_ohe[['Lag1', 'Lag2']])[1:]
y = np.array(weekly_ohe[['Direction']])[1:]
# first_obs = True
# X_train, X_test = [], []
# y_train, y_test = [], []
# for train_index, test_index in loocv.split(weekly_ohe[['Lag1', 'Lag2']]):
#     if first_obs:
#         train_indx_first = train_index
#         test_indx_first = test_index
#         first_obs = False
#     else:
#         print("TRAIN:", train_index, "TEST:", test_index)
#         # X_train, X_test = np.array(weekly_ohe[['Lag1', 'Lag2']])[train_index], np.array(weekly_ohe[['Lag1', 'Lag2']])[test_index]
#         # y_train, y_test = np.array(weekly_ohe['Direction'])[train_index], np.array(weekly_ohe['Direction'])[test_index]
#         # print(X_train, X_test, y_train, y_test)
#         # log_reg.fit(X=X_train, y=y_train)
#         # y_pred = log_reg.predict(X_test)
#         # scores += [log_reg.score(X_test, y_test)]
#         X_train += [np.array(weekly_ohe[['Lag1', 'Lag2']])[train_index]]
#         X_test +=  [np.array(weekly_ohe[['Lag1', 'Lag2']])[test_index]]
#         y_train += [np.array(weekly_ohe['Direction'])[train_index]]
#         y_test += [np.array(weekly_ohe['Direction'])[test_index]]

# print(np.mean(scores))
log_reg_no_first_obs = LogisticRegression()
log_reg_no_first_obs.fit(X=X, y=y)



# (c) Use the model from (b) to predict the direction of the first observation.
# You can do this by predicting that the first observationwill go up
#  if P(Direction = "Up"|Lag1, Lag2) > 0.5.
# Was this observation correctly classified?
prob_up = log_reg.predict_proba(X=np.array(weekly_ohe[['Lag1', 'Lag2']])[:1])
classf = 1 if prob_up[0][0] > 0.5 else 0
print(f"Results \n --> real: {weekly_ohe['Direction'][0]}  Vs  predicted: {classf}")
# log_reg_first_obs = LogisticRegression()
# log_reg_first_obs.fit(np.array(weekly_ohe[['Lag1', 'Lag2']])[train_indx_first], y=np.array(weekly_ohe['Direction'])[train_indx_first])
# log_reg_first_obs.predict_proba(np.array(weekly_ohe[['Lag1', 'Lag2']])[test_indx_first])




# (d) Write a for loop from i = 1 to i = n, where n is the number of
# observations in the data set, that performs each of the following
# steps:
# i. Fit a logistic regression model using all but the ith observation to predict Direction using Lag1 and Lag2.

# ii. Compute the posterior probability of the market moving up
# for the ith observation.

# iii. Use the posterior probability for the ith observation in order
# to predict whether or not the market moves up.

# iv. Determine whether or not an error was made in predicting
# the direction for the ith observation. If an error was made,
# then indicate this as a 1, and otherwise indicate it as a 0.




# (e) Take the average of the n numbers obtained in (d)iv in order to
# obtain the LOOCV estimate for the test error. Comment on the
# results.


Results 
 --> real: 0  Vs  predicted: 0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


In [184]:
weekly_ohe['Direction'][0]

0

In [180]:
# np.array([1,1]).reshape(-1,1)
# np.array(weekly_ohe[['Lag1', 'Lag2']])[:1]
prob_up[0][0]

0.42940147928293726